In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
books = pd.read_csv("/kaggle/input/book-recommendation-dataset/Books.csv", low_memory=False)
users = pd.read_csv("/kaggle/input/book-recommendation-dataset/Users.csv")
ratings = pd.read_csv("/kaggle/input/book-recommendation-dataset/Ratings.csv")

In [ ]:
books.head()

In [ ]:
users.head()

In [ ]:
books.info()

In [ ]:
users.info()

In [ ]:
ratings.info()

In [ ]:
ratings.head()

In [ ]:
print(books.shape)
print(users.shape)
print(ratings.shape)

In [ ]:
books.isnull().sum()

In [ ]:
users.isnull().sum()

In [ ]:
ratings.isnull().sum()

In [ ]:
books.duplicated().sum()

In [ ]:
users.duplicated().sum()

In [ ]:
ratings.duplicated().sum()

In [ ]:
books.columns

In [ ]:
users.columns

In [ ]:
ratings.columns

In [ ]:
# displaying top 50 books with highest average ratings, with ratings > 250)

In [ ]:
ratings_with_name = ratings.merge(books, on='ISBN')

ratings_with_name


In [ ]:
num_rating_df = ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
# grp by book title islia bcz many isbn no. for harry potter book, and some other books also have such issues 

In [ ]:
num_rating_df.rename(columns = {'Book-Rating' : 'num_ratings'} , inplace=True)
num_rating_df

In [ ]:
avg_rating_df = ratings_with_name.groupby('Book-Title')['Book-Rating'].mean().reset_index()
avg_rating_df.rename(columns={'Book-Rating': 'avg_ratings'}, inplace=True)
avg_rating_df

In [ ]:
popular_df = num_rating_df.merge(avg_rating_df, on='Book-Title')
popular_df

In [ ]:
popular_df[popular_df['num_ratings']>=250].sort_values('avg_ratings',ascending= False)

In [ ]:
# Collaborative filtering based recommender system
x=ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 200
users_with_more_than_200ratings = x[x].index
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(users_with_more_than_200ratings)]
filtered_rating.groupby('Book-Title').count()['Book-Rating']

In [ ]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating'] >= 50
books_with_more_than_50_ratings = y[y].index
books_with_more_than_50_ratings

In [ ]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(books_with_more_than_50_ratings)]
pt = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')
pt.fillna(0,inplace=True)
pt

In [ ]:
# each book here is a vector in 810 dimensional space, so we will find similaar vectors using cosin similarity
# finding similarity btw 2 non zero vectors by finding similarity between them.
from sklearn.metrics.pairwise import  cosine_similarity
cosine_similarity(pt)

In [ ]:
# 706 books ka euclidian distance nikala h 706 books k sath, hr book ka hr dusri book k sath.
similarity_score = cosine_similarity(pt)
similarity_score

In [ ]:
# now we will make a function named recommend, in which we pass a book and it will recommend 5 books in return.
def recommend(book_name):
    # fetch index of book
    index = np.where(pt.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_score[index])), key = lambda x:x[1], reverse = True)[1:6]
    suggestions = []
    for i in similar_items:
        suggestions.append(pt.index[i[0]])
        
    
    return suggestions

In [ ]:
recommend('1984')

In [ ]:
recommend('Message in a Bottle')